In [7]:
import numpy as np
import cv2 
import os 
import glob
import xml.etree.ElementTree as ET

In [8]:
# extract all the bounding boxes for one image 
def getBoxes(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    im_bboxes = []
    filename_ext = root.find('filename').text 
    dot_index  = filename_ext.find(".")
    filename = filename_ext[:dot_index]
    # Iterating over all 'object' elements (each representing one bounding box)
    for obj in root.findall('object'):
        label = obj.find('name').text  # Get the object label
        # Get bounding box coordinates
        bbox = obj.find('bndbox')
        if bbox is not None:
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)
        else:
            print(f"Error: Bounding box not found in this object {obj}, {filename}.")
        
        # Store the label and bounding box coordinates
        im_bboxes.append((filename, label, xmin, ymin, xmax, ymax))
    
    return im_bboxes

# Function to crop the image based on bounding box coordinates
def crop_image(image, xmin, ymin, xmax, ymax):
    # Crop the image using NumPy slicing (ymin:ymax, xmin:xmax)
    cropped_image = image[ymin:ymax, xmin:xmax]
    return cropped_image

In [9]:
def saveNormIm(image, im_dir, im_name):

    # make path if not exist 
    if not os.path.exists(im_dir):
        os.makedirs(im_dir)

    # Convert the image to float type for precision
    image = image.astype(np.float32)

    # Calculate the mean and standard deviation of the pixel intensities
    mean, std = np.mean(image), np.std(image)

    # Apply Z-normalization
    z_normalized_image = (image - mean) / std

    # Rescale the result back to a typical range (optional, if needed for display)
    # z_normalized_image = (z_normalized_image - np.min(z_normalized_image)) / (np.max(z_normalized_image) - np.min(z_normalized_image)) * 255

    # Convert the image back to uint8 type (if needed for saving/displaying)
    z_normalized_image = np.clip(z_normalized_image, 0, 255).astype(np.uint8)

    im_dir_and_name = f"{im_dir}{im_name}"    
    # Save or display the Z-normalized image
    cv2.imwrite(im_dir_and_name, z_normalized_image)


In [10]:
# assumes xml and jpeg/jpg file has the same name and in the same folder
def massCrop(srcXML, dest):
    
    # make path if not exist 
    if not os.path.exists(dest):
        os.makedirs(dest)

    xml_paths = glob.glob(srcXML)
    
    # for each xml/image extract [(lable, bnbox),...], then crop and save with UID 
    for xml_path in xml_paths:
        
        # find path of corresponding image 
        dot_index = xml_path.find(".")
        im_name = xml_path[:dot_index]
        im_path = glob.glob( im_name +".j*" ) 
        #print(im_path[0])
        
        if len(im_path) == 1:
            image = cv2.imread(im_path[0])
        else: 
            print("more than one image found corresponding to name!")
            print("exiting loop")
            break
        
        bboxes = getBoxes(xml_path)
        #print(bboxes)

        # crop and save for each instance 
        for (filename, label, xmin, ymin, xmax, ymax) in bboxes:
            cropped = crop_image(image, xmin, ymin, xmax, ymax )
            # TODO: check if jpg and jpeg are the same thing and conversion is ok 
            ID = f"{filename}_{xmin}_{ymin}_{xmax}_{ymax}_{label}.jpg"
            cropped_fileName = f"{dest}{ID}"
            
            if cropped is None:
                print(f"Error: Image not loaded correctly {cropped_fileName}")
            elif cropped.size == 0:
                print(f"Error: The image is empty after processing. {cropped_fileName}")
            else:
                # Define the uniform size 
                uniform_size = (64, 64)
                # Resize the image
                resized_image = cv2.resize(cropped, uniform_size)
                
                norm_file_path = f"{dest}/norm/"
                saveNormIm(resized_image, norm_file_path, ID)
                
                cv2.imwrite(cropped_fileName, resized_image)
    print("All images processed and saved!")

massCrop("greyAndEquPalm/test/*.xml", "singleTree/test/")
massCrop("greyAndEquPalm/train/*.xml", "singleTree/train/")



Error: The image is empty after processing. singleTree/test/ck2seqdja9nig07572xulcw2m_0_3000_0_3000_Palm.jpg
All images processed and saved!
Error: Bounding box not found in this object <Element 'object' at 0x14785e9d0>, ck2hhygv7ztus0757m18z1lqd.
Error: Bounding box not found in this object <Element 'object' at 0x1478fba10>, ck2ej1sehu1pn07946turgr8z.
Error: Bounding box not found in this object <Element 'object' at 0x1478fb150>, ck2ej1sehu1pn07946turgr8z.
All images processed and saved!


From investigations below, imread's default flags are read in as bgr, need to use flags IMREAD_GRAYSCALE

In [11]:
investigate = False
if investigate: 
    im_path = "/Users/catherineli/Library/CloudStorage/OneDrive-UniversityofCapeTown/2.CSC2005Z/code-related/greyAndEquPalm/test/ck2euxc9kxgvm07486g2d5pid.jpg"
    xml_path = "/Users/catherineli/Library/CloudStorage/OneDrive-UniversityofCapeTown/2.CSC2005Z/code-related/greyAndEquPalm/test/ck2euxc9kxgvm07486g2d5pid.xml"
    im = cv2.imread(im_path)

    # check if successfully read 
    if im is not None:
        
        #turn grey and equalize 
        #when turing grey scale, the rabe is [0,255]

        bboxes = getBoxes(xml_path)
        
        # crop and save for each instance 
        for (filename, label, xmin, ymin, xmax, ymax) in bboxes:
            cropped = crop_image(im, xmin, ymin, xmax, ymax )
            # TODO: check if jpg and jpeg are the same thing and conversion is ok 
            ID = f"{filename}_{xmin}_{ymin}_{xmax}_{ymax}_{label}.jpg"
            cropped_fileName = f"{"/Users/catherineli/Library/CloudStorage/OneDrive-UniversityofCapeTown/2.CSC2005Z/code-related/singleTree/"}{ID}"
            
            if cropped is None:
                print(f"Error: Image not loaded correctly {cropped_fileName}")
            elif cropped.size == 0:
                print(f"Error: The image is empty after processing. {cropped_fileName}")
            else:
                # Define the uniform size 
                uniform_size = (128, 128)
                # Resize the image
                resized_image = cv2.resize(cropped, uniform_size)
                single_channel_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2GRAY)
                print("dimesions of before conversion ", resized_image.ndim)
                print("dimesions of after conversion ",single_channel_image.ndim)
                ID2 = f"{filename}_{xmin}_{ymin}_{xmax}_{ymax}_{label}_norm.jpg"
                norm_file_path = f"{"/Users/catherineli/Library/CloudStorage/OneDrive-UniversityofCapeTown/2.CSC2005Z/code-related/singleTree/"}"
                
                saveNormIm(single_channel_image, norm_file_path, ID2)
                cv2.imwrite(cropped_fileName, single_channel_image)
        
        #print(grey_im.ndim)
        #print(grey_im)

        output_path = "/Users/catherineli/Library/CloudStorage/OneDrive-UniversityofCapeTown/2.CSC2005Z/test.jpg"


    else:
        print(f"Error reading image {im_path}")

In [12]:
# querying the images 
# imN_path = "/Users/catherineli/Library/CloudStorage/OneDrive-UniversityofCapeTown/2.CSC2005Z/code-related/singleTree/ck2euxc9kxgvm07486g2d5pid_0_2492_267_2848_Palm_norm.jpg"
# im_path = "/Users/catherineli/Library/CloudStorage/OneDrive-UniversityofCapeTown/2.CSC2005Z/code-related/singleTree/ck2euxc9kxgvm07486g2d5pid_0_2492_267_2848_Palm.jpg"
# im = cv2.imread(im_path)
# imN = cv2.imread(imN_path)

# grey_im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
# grey_imN = cv2.cvtColor(imN, cv2.COLOR_RGB2GRAY)

# # Display the image in a window
# #cv2.imshow('Image Window', grey_im)


# #cv2.waitKey(0)  # Wait for a key press
# #cv2.destroyAllWindows()  # Close all OpenCV windows

# print(im)
# print()
# print(imN)
# print()
# #print(grey_im)
# print()
# #print(grey_imN.ndim)

In [13]:
!ls -l singleTree/test/*.j* | wc -l
!ls -l singleTree/train/*.j* | wc -l
# total count of trees is 13 067, four is not in good format, explained below 

    2669
   10398


Error: The image is empty after processing. singleTree/test//ck2seqdja9nig07572xulcw2m_0300003000Palm.jpg
- the above has really wierd coordinates x-axis(0,0) and y-axis (3000,3000)

Error: Bounding box not found in this object <Element 'object' at 0x11f71b4c0> ck2hhygv7ztus0757m18z1lqd.
- the outline looks like the following 
<object>
        <name>Tree</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <polygon>
            <x1>293</x1>
                <y1>2989</y1>
            <x2>0</x2>
                <y2>2989</y2>
            <x3>0</x3>
                <y3>2489</y3>
            <x4>296</x4>
                <y4>2491</y4>
            </polygon>
        </object>
    <object>


Error: Bounding box not found in this object <Element 'object' at 0x11f71b8d0>, train/ck2ej1sehu1pn07946turgr8z.
Error: Bounding box not found in this object <Element 'object' at 0x11f719080>, train/ck2ej1sehu1pn07946turgr8z.
- the outline looks like the following
- line 31

<object>
    <name>Palm</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <polygon>
        <x1>306</x1>
            <y1>403</y1>
        <x2>306</x2>
            <y2>734</y2>
        <x3>644</x3>
            <y3>734</y3>
        <x4>644</x4>
            <y4>406</y4>
        </polygon>
</object>

- line 107

<object>
    <name>Palm</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <polygon>
        <x1>276</x1>
            <y1>896</y1>
        <x2>279</x2>
            <y2>1198</y2>
        <x3>610</x3>
            <y3>1198</y3>
        <x4>610</x4>
            <y4>893</y4>
        </polygon>
    </object>



